In [ ]:
import os
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# functions

In [ ]:
def compute_RMSE(diff):
    if len(diff)>0:
        return np.sqrt(np.mean(diff.values**2))
        
def compute_MAE(diff):
    if len(diff)>0:
        return np.mean(np.abs(diff.values))

def plot_errors(train_data,test_data):
    test_diff = test_data['outofsample_error']
    train_diff = train_data['insample_error']
    
    test_plot = test_data[['date','spot_price','rho','v0']].copy()
    test_plot = test_plot.reset_index().set_index('date')
    test_plot['MAE'] = test_diff.resample('D').apply(compute_RMSE)
    test_plot['RMSE'] = test_diff.resample('D').apply(compute_RMSE)
    test_plot = test_plot.reset_index()
    test_plot = test_plot.drop_duplicates(subset=['date'],keep='last').set_index('date').drop(columns='calculation_date')
    
    train_plot = train_data[['date','spot_price','rho','v0']].copy()
    train_plot = train_plot.reset_index().set_index('date')
    train_plot['MAE'] = train_diff.resample('D').apply(compute_RMSE)
    train_plot['RMSE'] = train_diff.resample('D').apply(compute_RMSE)
    train_plot = train_plot.reset_index()
    train_plot = train_plot.drop_duplicates(subset=['date'],keep='last').set_index('date').drop(columns='calculation_date')
    trainx = pd.date_range(start=min(train_data.index),end=max(train_data.index),periods=train_plot.shape[0])
    train_plot.index = trainx

    
    testx = pd.date_range(start=min(test_data.index),end=max(test_data.index),periods=test_plot.shape[0])
    test_plot.index = testx
    
    
    fig,axs = plt.subplots(max(len(train_plot.columns),len(test_plot.columns)),figsize=(10,10),sharex=True)
    for i,col in enumerate(train_plot.columns):
        axs[i].plot(train_plot[col],color='green',label='in-sample')
        axs[i].set_title(col.replace('_',' '))
        axs[i].legend()
    for i,col in enumerate(test_plot.columns):
        axs[i].plot(test_plot[col],color='purple',label='out-of-sample')
        axs[i].set_title(col.replace('_',' '))
        axs[i].legend()
    plt.show()

# loading model

In [ ]:
from model_settings import ms
root = Path().resolve().parent.parent
models_dir = os.path.join(root,ms.trained_models)
models = pd.Series([f for f in os.listdir(models_dir) if f.find('Legacy')==-1])
for i,m in enumerate(models):
    print(f"{i}     {m}")

In [ ]:
selected_model = models.iloc[0]
model_dir = os.path.join(models_dir,selected_model)
pickle = [f for f in os.listdir(model_dir) if f.endswith('.pkl')][0]
picke_dir = os.path.join(model_dir,pickle)
model = joblib.load(picke_dir)
model

In [ ]:
model['train_data']

In [ ]:
print('model attributes:\n')
for k in model.keys():
    print(k)

In [ ]:
for col in model['feature_set']:
    print(f"{col.replace("_"," ")}:",f"\n{model['test_data'][col].copy().squeeze().sort_values().drop_duplicates().reset_index(drop=True)}\n")
print()

In [ ]:
train_data = model['train_data'].copy()
test_data = model['test_data'].copy()
train_data['calculation_date'] = pd.to_datetime(train_data['calculation_date'],format='mixed')
test_data['calculation_date'] = pd.to_datetime(test_data['calculation_date'],format='mixed')
test_data = test_data.set_index('calculation_date').sort_index()
train_data = train_data.set_index('calculation_date').sort_index()
plot_errors(train_data,test_data)
print(model['feature_set'])

# retraining

In [ ]:
from convsklearn import convsklearn
help(convsklearn)

In [ ]:
retraining_frequency = 30 #days
test_dates = model['test_dates']
full_dataset = model['dataset']
all_dates = full_dataset['date'].drop_duplicates().sort_values().reset_index(drop=True)
all_dates

In [ ]:
retraining_dates = len(test_dates)//retraining_frequency
retraining_dates = test_dates.iloc[np.linspace(0,len(test_dates)-1,retraining_dates,dtype=int)]
retraining_dates = retraining_dates.reset_index(drop=True)

def retrain(date):
    development_dates = all_dates[all_dates<=date]
    subset_test_dates = all_dates[~all_dates.isin(development_dates)].iloc[:retraining_frequency]
    retrainer = convsklearn()
    retrainer.excluded_features += model['excluded_features']
    retrainer.target_name = model['target_name']
    retrainer.load_data(full_dataset)
    retrainer.preprocess_data(development_dates, subset_test_dates,plot=False)
    retrainer.run_dnn(print_details=False)
    retrainer.test_prediction_accuracy()
    print(date)
    return (date,retrainer)

from joblib import Parallel, delayed
models = Parallel()(delayed(retrain)(d) for d in retraining_dates)

In [ ]:
stop yo

In [ ]:
retraining_dates = [d for d in models.keys()]
cols = ['cpu','isMAE','isRMSE','osMAE','osRMSE']
df = pd.DataFrame(np.tile(np.nan,(len(retraining_dates),len(cols))),columns=cols,index=retraining_dates)

for i,row in df.iterrows():
    m = models[i]
    df.at[i,'cpu'] = m['dnn_runtime']
    df.at[i,'isMAE'] = compute_MAE(m['train_data']['insample_error'])
    df.at[i,'isRMSE'] = compute_RMSE(m['train_data']['insample_error'])
    df.at[i,'osMAE'] = compute_MAE(m['test_data']['outofsample_error'])
    df.at[i,'osRMSE'] = compute_RMSE(m['test_data']['outofsample_error'])


print(df)

In [ ]:
from plotters import PlotCols
PlotCols(df)